In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import catboost
import tensorflow as tf

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

In [ ]:
tf.test.gpu_device_name()

In [ ]:
import os
df=pd.read_json('../input/whats-cooking-kernels-only/train.json', orient='records', dtype={"id":int, "cuisine":str,"ingredients":list})
df_test=pd.read_json('../input/whats-cooking-kernels-only/test.json', orient='records', dtype={"id":int, "ingredients":list})

#### Датасет: анализ и подготовка

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['cuisine'].unique(), len(df['cuisine'].unique())

In [ ]:
sns.countplot(y='cuisine', data=df)
plt.show()

In [ ]:
lists = [df['ingredients'].values[i] for i in range(len(df))]
unique_ingredients = list(set(list(np.concatenate(lists))))
d = dict(zip(np.arange(len(unique_ingredients)), unique_ingredients))
inv_d = dict(zip(unique_ingredients, np.arange(len(unique_ingredients))))

In [ ]:
sorted(inv_d.items())

In [ ]:
dict(sorted(inv_d.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import string
from nltk.corpus import stopwords
import re

In [ ]:
from tqdm import tqdm_notebook
total_ingredients_list = []
tq = tqdm_notebook(total=df.shape[0])
for ing in df['ingredients']:
    total_ingredients_list = total_ingredients_list + ing
    tq.update(1)
tq.close()

In [ ]:
chars = re.escape(string.punctuation)
chars

In [ ]:
# - убираем знаки препинания, цифры, "oz", пробелы в начале строки и пишем все строчными буквами
clean_ingredients_list = [re.sub(r'['+chars+']', '', 
                                 re.sub('[0-9]+','', c.replace("oz",""))).strip(' ').lower()
                    for c in total_ingredients_list]

In [ ]:
len(np.unique(total_ingredients_list))

In [ ]:
sorted(np.unique(clean_ingredients_list))[:15]

In [ ]:
len(np.unique(clean_ingredients_list))

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

In [ ]:
tq = tqdm_notebook(total=len(clean_ingredients_list))
for i, ingredients in enumerate(clean_ingredients_list):
    cleaned_ingredients = [c for c in ingredients.split(' ') if c not in stop_words]
    cleaned_ingredients = (' '.join(cleaned_ingredients)).strip(' ')
    clean_ingredients_list[i] = cleaned_ingredients
    tq.update(1)
tq.close()
clean_unique_ingredients_list = [c for c in list(np.unique(clean_ingredients_list)) if len(c)>0]  

In [ ]:
len(clean_unique_ingredients_list)

In [ ]:
# превращаем список ингридиетов в дамми-переменные

In [ ]:
classes_dict = {'greek':0, 'southern_us':1, 'filipino':2, 'indian':3, 'jamaican':4,
                'spanish':5, 'italian':6, 'mexican':7, 'chinese':8, 'british':9, 'thai':10,
                'vietnamese':11, 'cajun_creole':12, 'brazilian':13, 'french':14, 'japanese':15,
                'irish':16, 'korean':17, 'moroccan':18, 'russian':19}

ingredients_encoded = np.zeros((df.shape[0], len(clean_unique_ingredients_list)+2), dtype=np.uint8)
tq = tqdm_notebook(total=df.shape[0])
for i in range(df.shape[0]):
    ingredients_encoded[i,0] = df['id'].values[i]
    ingredients_encoded[i,1] = classes_dict[df['cuisine'][i]]
    clean_recipe = [re.sub(r'['+chars+']', '',
                           re.sub('[0-9]+','', c.replace("oz",""))).strip(' ').lower()
                    for c in df['ingredients'][i]]
    for k, ingredients in enumerate(clean_recipe):
        cleaned_ingredients = [c for c in ingredients.split(' ') if c not in stop_words]
        cleaned_ingredients = (' '.join(cleaned_ingredients)).strip(' ')
        clean_recipe[k] = cleaned_ingredients
        
    clean_recipe = [c for c in clean_recipe if len(c)>0]
    for ingredient in clean_recipe:
        ingredients_encoded[i,clean_unique_ingredients_list.index(ingredient)+2]= 1
        
    tq.update(1)
    
tq.close()

In [ ]:
ingredients_encoded_test = np.zeros((df_test.shape[0], len(clean_unique_ingredients_list)+2), dtype=np.uint8)

tq = tqdm_notebook(total=df_test.shape[0])
for i in range(df_test.shape[0]):
    ingredients_encoded_test[i,0] = df_test['id'].values[i]
    ingredients_encoded_test[i,1] = 0
    clean_recipe = [re.sub(r'['+chars+']', '',
                           re.sub('[0-9]+','', c.replace("oz",""))).strip(' ').lower()
                    for c in df_test['ingredients'][i]]
    for k, ingredients in enumerate(clean_recipe):
        cleaned_ingredients = [c for c in ingredients.split(' ') if c not in stop_words]
        cleaned_ingredients = (' '.join(cleaned_ingredients)).strip(' ')
        clean_recipe[k] = cleaned_ingredients
    clean_recipe = [c for c in clean_recipe if len(c)>0]
    for ingredient in clean_recipe:
        if ingredient not in clean_unique_ingredients_list:
            continue
        ingredients_encoded_test[i,clean_unique_ingredients_list.index(ingredient)+2]= 1
        
    tq.update(1)
    
tq.close()

In [ ]:
data_test = pd.DataFrame(data=ingredients_encoded_test[:,1:])
data_test

In [ ]:
data = pd.DataFrame(data=ingredients_encoded[:,1:])
data

In [ ]:
data_gb = data.groupby([0]).sum()

In [ ]:
sns.heatmap(data_gb.loc[:,data_gb.sum(0)>1000], )

In [ ]:
data_gb.sum(0)[data_gb.sum(0)<2]

In [ ]:
len(data_test.sum(0)[data_test.sum(0)<2])

In [ ]:
data = data.loc[:,data.sum(0)>1]

In [ ]:
data

#### Датасет: трейн - тест

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = data[data.columns[0]]
X = data[data.columns[1:]]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y, test_size=0.2)

In [ ]:
import timeit
def prtime(start, stop):
    temp = stop - start
    hours = temp//3600
    temp = temp - 3600*hours
    minutes = temp//60
    seconds = temp - 60*minutes
    print('%d:%d:%d' %(hours,minutes,seconds))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

start = timeit.default_timer()
rf_best = RandomForestClassifier(bootstrap=True, 
                                 class_weight='balanced',
                                 criterion='gini', 
                                 max_depth=100, 
                                 max_features='auto',
                                 max_leaf_nodes=None, 
                                 min_impurity_decrease=0.0,
                                 min_impurity_split=None, 
                                 min_samples_leaf=1,
                                 min_samples_split=2, 
                                 min_weight_fraction_leaf=0.0,
                                 n_estimators=400, 
                                 n_jobs=8, 
                                 oob_score=False,
                                 random_state=None, verbose=0, warm_start=False)
# rf_best.fit(X_train,y_train)
stop = timeit.default_timer()
prtime(start, stop)

In [ ]:
from sklearn.metrics import accuracy_score
# y_pred = rf_best.predict(X_test)
# accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.neural_network import MLPClassifier
start = timeit.default_timer()
clf_best = MLPClassifier(solver='adam',
                         activation='relu',
                         early_stopping=True, 
                         random_state=3,
                         max_iter=500,
                         verbose=True,
                         alpha=1e-08,
                         beta_1=0.05, 
                         beta_2=0.4, 
                         epsilon=1e-09)

# clf_best.fit(X, y)
stop = timeit.default_timer()
prtime(start, stop)

In [ ]:
from sklearn.metrics import accuracy_score
# y_pred = clf.predict(X_test)
# accuracy_score(y_test,y_pred)

In [ ]:
# from sklearn.linear_model import LogisticRegression

# start = timeit.default_timer()
# clf_best = LogisticRegression(penalty='elasticnet', 
#                               l1_ratio = 0.8,
#                               fit_intercept = True,
#                               class_weight = 'balanced',
#                               solver = 'saga',
#                               multi_class = 'multinomial'
#                              )
# clf_best.fit(X, y)
# stop = timeit.default_timer()
# prtime(start, stop)

In [ ]:
from sklearn.linear_model import LogisticRegression

start = timeit.default_timer()
clf_best = LogisticRegression(penalty='l2', 
                              fit_intercept = True,
                              class_weight = 'balanced',
                              solver = 'newton-cg',
                              multi_class = 'multinomial'
                             )
clf_best.fit(X, y)
stop = timeit.default_timer()
prtime(start, stop)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = clf_best.predict(X_test)
accuracy_score(y_test,y_pred)

In [ ]:
X_sub = data_test[data_test.columns[1:]]
X_sub = X_sub.loc[:, X.columns]

In [ ]:
y_sub = clf_best.predict(X_sub)
y_pred_str = [list(classes_dict.keys())[list(classes_dict.values()).index(c)] for c in y_sub]

In [ ]:
subm = pd.DataFrame({'id':df_test['id'], 'cuisine':y_pred_str})

In [ ]:
len(np.unique(subm['cuisine']))

In [ ]:
subm

In [ ]:
subm.to_csv('submission.csv',  index=False)